<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<div align='center'>
###  Felipe González - 201273534-3 
### Ignacio Tampe - 201573514-k 
</div>
<hr style="height:2px;border:none"/>

## 1. Back-propagation (BP) from *Scratch*


In [ ]:
class RedNeuronal:
    def __init__(self, num_entrada, tam_capas, num_salida ) #tamp_capas es una lista 

def feed_forward(self,input):
    nueva_capa = []
    for neurona in self.neuronas:
        outputs.append(neurona.calcular_salida())
    return nueva_capa


## 1. Referencias
